# Homework 2

In [10]:
!pip install requests PyPDF2 gdown
!pip install 'markitdown[pdf]'
!pip install langchain_mcp_adapters langchain_google_genai langchain-openai
!pip install -U langchain langchain-community langchain-google-genai langchain-mcp-adapters

In [11]:
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')

In [12]:
import os
import gdown

folder_id = "1adYKq7gSSczFP3iikfA8Er-HSZP6VM7D"
folder_url = f"https://drive.google.com/drive/folders/{folder_id}"

output_dir = "downloaded_cvs"
os.makedirs(output_dir, exist_ok=True)

gdown.download_folder(
    url=folder_url,
    output=output_dir,
    quiet=False,
    use_cookies=False
)

Retrieving folder contents


Processing file 1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp CV_1.pdf
Processing file 16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs CV_2.pdf
Processing file 15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr CV_3.pdf
Processing file 1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk CV_4.pdf
Processing file 1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C CV_5.pdf


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp
To: /content/downloaded_cvs/CV_1.pdf
100%|██████████| 147k/147k [00:00<00:00, 52.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs
To: /content/downloaded_cvs/CV_2.pdf
100%|██████████| 75.1k/75.1k [00:00<00:00, 35.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr
To: /content/downloaded_cvs/CV_3.pdf
100%|██████████| 72.0k/72.0k [00:00<00:00, 36.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk
To: /content/downloaded_cvs/CV_4.pdf
100%|██████████| 73.3k/73.3k [00:00<00:00, 32.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C
To: /content/downloaded_cvs/CV_5.pdf
100%|██████████| 97.9k/97.9k [00:00<00:00, 2.10MB/s]
Download complete

['downloaded_cvs/CV_1.pdf',
 'downloaded_cvs/CV_2.pdf',
 'downloaded_cvs/CV_3.pdf',
 'downloaded_cvs/CV_4.pdf',
 'downloaded_cvs/CV_5.pdf']

In [13]:
# =====================================================
#  Load and display all CV PDFs in order
# =====================================================
import os
from markitdown import MarkItDown

cv_dir = "downloaded_cvs"

md = MarkItDown(enable_plugins=False)

pdf_files = sorted(
    [f for f in os.listdir(cv_dir) if f.lower().endswith(".pdf")],
    key=lambda x: int("".join(filter(str.isdigit, x)))  # CV_1.pdf → 1
)

all_cvs = []

for pdf_name in pdf_files:
    pdf_path = os.path.join(cv_dir, pdf_name)
    result = md.convert(pdf_path)

    all_cvs.append({
        "file": pdf_name,
        "text": result.text_content
    })

    print("=" * 80)
    print(f"📄 {pdf_name}")
    print("=" * 80)
    print(result.text_content)
    print("\n\n")


📄 CV_1.pdf
|     |     |     |     | John         |           | Smith        |                   |     |     |
| --- | --- | --- | --- | ------------ | --------- | ------------ | ----------------- | --- | --- |
|     |     |     |     | Marketing    |           | Professional |                   |     |     |
|     |     |     |     | + Singapore, | Singapore |              | (cid:209) Kowloon |     |     |
Experience
|                |                  |     |          |                     |              |            |     | 2020 – | Present |
| -------------- | ---------------- | --- | -------- | ------------------- | ------------ | ---------- | --- | ------ | ------- |
| Engineer,      | ByteDance        |     |          |                     |              |            |     |        |         |
| • Worked       | in a fast-paced, |     | global   | technology          | environment. |            |     |        |         |
| • Collaborated | across           |     | teams to | sup

In [14]:
import asyncio
import json
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
for tool in mcp_tools:
    print(tool.name)
    print(tool.description)
    print(tool.args)
    print("\n\n------------------------------------------------------\n\n")

search_facebook_users
Search for Facebook users by display name (supports partial and fuzzy matching).

Args:
    q: Search query string (case-insensitive, matches any part of display name)
       Examples: "John", "john smith", "Smith"
    limit: Maximum number of results to return (default: 20, max: 20)
    fuzzy: Enable fuzzy matching if exact search returns no results (default: True)

Returns:
    List of user dictionaries, each containing:
    - id (int): Unique Facebook user ID for use with get_facebook_profile()
    - display_name (str): User's Facebook display name (may differ from legal name)
    - city (str): Current city of residence
    - country (str): Country of residence
    - match_type (str): "exact" or "fuzzy" (indicates search method used)
    
    Returns empty list [] if no matches found.

Example:
    search_facebook_users("Alex Chan", limit=5)
    → [{"id": 123, "display_name": "Alex Chan", "city": "Hong Kong", "country": "Hong Kong", "match_type": "exact"}]
    

In [15]:
import os
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient

# ---------------------------
# 1. Define a local tool
# ---------------------------
@tool
def say_hello(name: str) -> str:
    """Say hello to a person by name."""
    return f"Hello, {name}! 👋"

# ---------------------------
# 2. Load MCP tools + merge
# ---------------------------
client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
tools = mcp_tools + [say_hello]

# ---------------------------
# 3. Initialize Gemini (tool-enabled) or deepseek
# ---------------------------
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GEMINI_VERTEX_API_KEY,
    vertexai=True,
    temperature=0
)

llm_with_tools = llm.bind_tools(tools)

# ---------------------------
# 4. Single-step invocation
# ---------------------------
query = "Say hello to Bao using tool, then search for someone named Alice on Facebook."

response = llm_with_tools.invoke([
    HumanMessage(content=query)
])

print(response)

content='' additional_kwargs={'function_call': {'name': 'search_facebook_users', 'arguments': '{"q": "Alice"}'}} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c9332-242c-7692-8e11-555550502468-0' tool_calls=[{'name': 'say_hello', 'args': {'name': 'Bao'}, 'id': 'cbd2a71e-ee59-4678-8e2b-d5329694e457', 'type': 'tool_call'}, {'name': 'search_facebook_users', 'args': {'q': 'Alice'}, 'id': '96cf5883-b188-4a34-aae9-54435f8521f4', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 2568, 'output_tokens': 13, 'total_tokens': 2581, 'input_token_details': {'cache_read': 0}}


In [16]:
# This block provides you some tests to get faminilar with our MCP server

#Test 1: Search Facebook users (exact match)
#await tools[0].ainvoke({'q': "Alex Chan", 'limit': 5})

#Test 2: Search Facebook users (fuzzy match with typo)
await tools[0].ainvoke({'q': "Alx Chn", 'limit': 5, 'fuzzy': True})

# # Test 3: Get Facebook profile
#await tools[1].ainvoke({'user_id': 123})

# # Test 4: Get Facebook mutual friends
#await tools[2].ainvoke({'user_id_1': 123, 'user_id_2': 456})

# # Test 5: Search LinkedIn people (exact match)
#await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5})

# # Test 6: Search LinkedIn people (fuzzy match with typo)
#await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5, 'fuzzy': True})

# # Test 7: Get LinkedIn profile
#await tools[4].ainvoke({'person_id': 456})

# Test 8: Get LinkedIn interactions
#await tools[5].ainvoke({'person_id': 456})

[{'type': 'text',
  'text': '[{"id":3,"display_name":"Alex Chan","city":"Hyderabad","country":"India","match_type":"fuzzy"},{"id":41,"display_name":"Alex Chan","city":"Shanghai","country":"China","match_type":"fuzzy"},{"id":79,"display_name":"Alex Chan","city":"London","country":"UK","match_type":"fuzzy"},{"id":106,"display_name":"Alex Chan","city":"Ho Chi Minh City","country":"Vietnam","match_type":"fuzzy"},{"id":117,"display_name":"Alex Chan","city":"Melbourne","country":"Australia","match_type":"fuzzy"}]',
  'id': 'lc_87ce7c3e-d6b4-4c31-aa80-66dc64d29725'}]

In [21]:
def evaluate(scores, groundtruth, threshold=0.5):
    """
    scores: list of floats in [0, 1], length = 5
    groundtruth: list of ints (0 or 1), length = 5
    """
    assert len(scores) == 5
    assert len(groundtruth) == 5

    correct = 0
    decisions = []

    for s, gt in zip(scores, groundtruth):
        pred = 1 if s > threshold else 0
        decisions.append(pred)
        if pred == gt:
            correct += 1

    final_score = correct / len(scores)

    return {
        "decisions": decisions,
        "correct": correct,
        "total": len(scores),
        "final_score": final_score
    }


In [28]:
import re
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage

async def agent_run_loop(cv_text):
    messages = [
        HumanMessage(content=f"""You are a professional Background Check Specialist.
        Your task is to verify the authenticity of the following CV by cross-referencing it with Social Media data (LinkedIn/Facebook).

        Verification Protocol:
        1. MANDATORY: Call search tools to find the candidate.
        2. MANDATORY: Retrieve the full profile to see specific years and companies.
        3. COMPARISON: Compare company names, job titles, start years, and end years.
        4. SCORING CRITERIA:
           - If all key experiences match: Final Score: 0.9
           - If there is a minor discrepancy: Final Score: 0.5
           - If a company is missing, years overlap suspiciously, or titles are fabricated: Final Score: 0.1

        You must end your response strictly with 'Final Score: X.X'.

        CV Content:
        {cv_text}""")
    ]

    for _ in range(3):
        response = llm_with_tools.invoke(messages)
        messages.append(response)

        if not response.tool_calls:
            break

        for tool_call in response.tool_calls:
            t_name = tool_call["name"]
            t_args = tool_call["args"]
            print(f"  [Executing Tool] {t_name}: {t_args}")

            target_tool = next((t for t in tools if t.name == t_name), None)
            if target_tool:
                observation = await target_tool.ainvoke(t_args)
                print(f"  [Tool Observation]: {str(observation)[:100]}...")
                messages.append(ToolMessage(content=str(observation), tool_call_id=tool_call["id"]))
            else:
                messages.append(ToolMessage(content="Error: Tool not found", tool_call_id=tool_call["id"]))

    return messages[-1].content

async def get_final_scores(cv_list):
    scores = []
    for cv in cv_list:
        print(f"\n--- Verifying: {cv['file']} ---")
        final_text = await agent_run_loop(cv['text'])

        match = re.search(r"Final Score:\s*(\d?\.\d+)", final_text)
        if match:
            s = float(match.group(1))
        else:
            s = 0.2 if "discrepancy" in final_text.lower() or "not found" in final_text.lower() else 0.8

        print(f">>> {cv['file']} Resulting Score: {s}")
        scores.append(s)
    return scores

In [29]:
scores = await get_final_scores(all_cvs)
groundtruth = [1, 1, 1, 0, 0]
result = evaluate(scores, groundtruth)
print(result)


--- Verifying: CV_1.pdf ---
  [Executing Tool] search_linkedin_people: {'q': 'John Smith', 'location': 'Singapore'}
  [Tool Observation]: [{'type': 'text', 'text': '[{"id":9,"name":"John Smith","headline":"Marketing Professional","industr...
  [Executing Tool] get_linkedin_profile: {'person_id': 9}
  [Tool Observation]: [{'type': 'text', 'text': '{"id":9,"name":"John Smith","headline":"Marketing Professional","city":"S...
>>> CV_1.pdf Resulting Score: 0.9

--- Verifying: CV_2.pdf ---
  [Executing Tool] search_linkedin_people: {'q': 'Minh Pham', 'location': 'Beijing, China'}
  [Tool Observation]: []...
  [Executing Tool] search_linkedin_people: {'q': 'Minh Pham', 'location': 'Hong Kong'}
  [Tool Observation]: [{'type': 'text', 'text': '[{"id":180,"name":"Minh Pham","headline":"Consulting Professional","indus...
  [Executing Tool] get_linkedin_profile: {'person_id': 2758}
  [Tool Observation]: [{'type': 'text', 'text': '{"id":2758,"name":"Minh Pham","headline":"Design Professional","cit